<div style="display: flex; justify-content: space-between; align-items: center; margin-bottom: 40px; margin-top: 0;">
    <div style="flex: 0 0 auto; margin-left: 0; margin-bottom: 0; margin-top: 0;">
        <img src="./pics/UCSD Logo.png" alt="UCSD Logo" style="width: 179px; margin-bottom: 0px; margin-top: 20px;">
    </div>
    <div style="flex: 0 0 auto; margin-left: auto; margin-bottom: 0; margin-top: 20px;">
        <img src="./pics/LANL-logo.png" alt="LANL Logo" style="width: 200px; margin-bottom: 0px;">
    </div>
    <div style="flex: 0 0 auto; margin-left: auto; margin-bottom: 0; margin-top: 20px;">
        <img src="./pics/prowess.png" alt="Prowess Logo" style="width: 200px; margin-bottom: 0px;">
    </div>
    <div style="flex: 0 0 auto; margin-left: auto; margin-bottom: 0; margin-top: 20px;">
        <img src="./pics/wildfire.png" alt="WildFire Logo" width="100"/>
    </div>
</div>

<h1 style="text-align: center; font-size: 48px; margin-top: 0;">Fire-Ready Forests Data Challenge</h1>

# Sprint 2: Generating a Treelist from ALS

In the previous sprint, you worked with a notebook exploring the FIA Database. In that notebook, we introduced a technique called **Individual Tree Detection**. If you need a refresher, you can revisit Sprint 1 - Module A and open the `fia_database.ipynb` notebook, specifically the *Preliminary Analysis* section.

We also provided a file, `ttops.csv`, which contains a list of trees derived from ALS data. In this module, we will demonstrate how this list is generated.

Instead of processing the entire dataset—which would require significant computation time—we will focus on a single `.laz` file. Running the full computation on this file should take approximately 20–25 minutes.

## Part 1 - Canopy Height Model

The first dataset we’ll extract from our ALS file is the **Canopy Height Model (CHM)**. The CHM represents the height of vegetation, such as trees, above the ground. To create it, we subtract the Digital Terrain Model (DTM)—which captures the bare ground elevation—from the Digital Surface Model (DSM). This allows us to measure the true height of vegetation in the landscape. Mathematically, it is expressed as: 

$$
CHM = DSM - DTM
$$



In [ ]:
# First, we import all the libraries
import pc_rasterize as pcr
import glob
import numpy as np
import requests
import pandas as pd
from dask.distributed import Client, LocalCluster, Lock
import rasterio
import matplotlib.pyplot as plt
from skimage.feature import peak_local_max
from pyproj import Transformer

To run our script successfully, we first need to download the file locally. Unfortunately, we can't load the file directly into memory from a URL, so we'll start by selecting one of the links from the *ALS Raw Files Download Links* file and saving it to our storage. Once you've completed this module, you can delete the file.

In [ ]:
url = "https://wifire-data.sdsc.edu/nc/s/nnSMqWfZAN6Cz6m/download?path=%2Fnew_data%2FRaw%20ALS%20and%20Detected%20Trees%2FALS%20data&files=USGS_LPC_CA_SierraNevada_B22_10SGJ2967.laz&downloadStartSecret=aik5h9wexsu"
local_als = "./USGS_LPC_CA_SierraNevada_B22_10SGJ2967.laz"

response = requests.get(url, stream=True)
with open(local_als, "wb") as file:
    for chunk in response.iter_content(chunk_size=8192):
        file.write(chunk)

print(f"Downloaded: {local_als}")

Now that we've downloaded the file, let's take a quick look at its key details.

In [ ]:
pcr.get_file_quickinfo('./USGS_LPC_CA_SierraNevada_B22_10SGJ2967.laz')

Let's proceed to build a pipeline for our file processing.

In [ ]:
als = glob.glob("./USGS_LPC_CA_SierraNevada_B22_10SGJ2967.laz") # We load our file

# Create a GeoBox grid specification with a 100m buffer around data
geobox = pcr.build_geobox(als, resolution=1., crs="5070")

# Build a lazy CHM raster
chm = pcr.rasterize(
    als,
    geobox,
    cell_func="max",
    # Set custom dask chunk-size
    chunksize=(1000, 1000),
    nodata=np.nan,
    pdal_filters=[
    {
        "type":"filters.range",
        "limits":"Classification[1:5]" # Keep points classified as 'vegetation' - https://desktop.arcgis.com/en/arcmap/latest/manage-data/las-dataset/lidar-point-classification.htm
    },
    {
      "type":"filters.outlier" # Remove outliers
    },
    {
        "type":"filters.hag_nn" # Compute Height Above Ground (HAG) using the nearest-neighbor method. 
    },
    {
        "type":"filters.ferry",
        "dimensions":"HeightAboveGround=>Z" # Move the computed HAG values into the `Z` coordinate for easier processing.  
    },
    {
      "type":"filters.outlier" # Remove outliers again
    },
    {
        "type": "filters.expression",
        "expression": "Z < 75" # Filters out points higher than 75m (unrealistic height in our context) 
    }
    ]
)
# A quick look to our xarray
chm

With our pipeline ready, we will proceed with the computation. The next line will take 10-20 minutes.

In [ ]:
with LocalCluster(n_workers=8, threads_per_worker=1, memory_limit='24GB') as cluster, Client(cluster) as client:
    chm_squeezed = chm.squeeze()  # We remove an extra dimension with no data
    chm_squeezed.rio.to_raster("./chm.tiff", tiled=True, lock=Lock("rio"))
print("Successfuly computed the CHM!")

**NOTE:** The source repository of the method that we applied in this notebook can be found [here](https://github.com/UM-RMRS/pc-rasterize).

## Part 2 - TreeTops

Ok, now that we have our CHM ready, we can proceed to generate our list of trees. But first, let's visualize our CHM raster file.

In [ ]:
chm_path = "./chm.tiff"
with rasterio.open(chm_path) as src:
    chm = src.read(1)  
    transform = src.transform  # Affine transform for georeferencing
    crs = src.crs  # Get raster Coordinate Reference System

In [ ]:
# Let's quickly visualize our raster
plt.figure(figsize=(10, 6))
plt.imshow(chm, cmap="viridis", interpolation="nearest")
plt.colorbar(label="Canopy Height (m)")
plt.title("Canopy Height Model")
plt.show()

Pretty cool right? 

With our CHM ready, it's time to move on to **tree detection**—extracting their heights and precise coordinates. Let’s get started!

In [ ]:
# Normalize CHM 
chm_norm = (chm - np.nanmin(chm)) / (np.nanmax(chm) - np.nanmin(chm))

# Locate tree tops using local maxima filtering
tree_tops = peak_local_max(chm_norm, min_distance=3, threshold_abs=0.1, exclude_border=False)

# Convert pixel coordinates to geospatial coordinates and extract heights
tree_points = []
for row, col in tree_tops:
    x, y = transform * (int(col), int(row))  
    height = chm[row, col] 
    tree_points.append((height, y, x)) 

df = pd.DataFrame(tree_points, columns=["HT", "y", "x"])

# Let´s look at the df
df.head()

In [ ]:
# Let's take a look at the distribution of tree heights
plt.figure(figsize=(10, 5))
plt.hist(df["HT"], bins=30, edgecolor="black", alpha=0.7)
plt.xlabel("Tree Height (HT)")
plt.ylabel("Frequency")
plt.title("Distribution of Tree Heights (HT)")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()

In [ ]:
# Convert to Lat/Lon system
if crs and crs.to_epsg() != 4326:
    transformer = Transformer.from_crs(crs.to_epsg(), "EPSG:4326", always_xy=True)
    df["x"], df["y"] = transformer.transform(df["x"].values, df["y"].values) 

# Save CSV
df.to_csv("./ttops.csv", index=False)
print(f"TreeTops saved to ./data/ttops.csv with {len(df)} points")

Success! Now you know how to generate a ttops file like the one you used in the previous sprint, and that you will continue to use in the following sprints. 

If you're interested in learning more about Individual Tree Detection (ITD) and the various libraries and software tools available for creating similar tree lists, check out the following resources:

- https://research.fs.usda.gov/treesearch/59063
- https://tgoodbody.github.io/lidRtutorial/06_its.html
- https://www.mathworks.com/help/lidar/ug/extraction-of-forest-metrics-and-individual-tree-attributes.html